In [1]:
from datetime import datetime
print(f'Päivitetty {datetime.now().date()} / Aki Taanila')

Päivitetty 2025-10-19 / Aki Taanila


# Annuiteettilainojen havainnollistaminen ja vertailu

Tässä muistiossa tarkastellaan ja vertaillaan kuukausittain lyhennettäviä annuitettilainoja. Tulokset kirjoitetaan Excel-tiedostoon pandas-paketin **to_excel**-funktiolla.  Kirjoittimena käytetään **xlsxwriter**ia.

Suorituksen jälkeen **laina.xlsx**-tiedosto löytyy samasta hakemistosta kuin tämä muistio. **Varoitus:** **Jos hakemistossa on aiemmin laina.xlsx-niminen tiedosto, niin se ylikirjoitetaan.

In [2]:
import pandas as pd
from datetime import date
import xlwings as xw
import annuiteetti  #oletus: annuiteetti.py-moduuli on tallennettu samaan kansioon tämän muistion kanssa

## Lähtötiedot

Täydennä seuraavaan soluun lainan lähtötiedot:
- korkokanta desimaalimuodossa; esimerkiksi ,0245 tarkoittaa 2,45 % korkokantaa.
- aika_kuukausina: aseta laina-ajaksi 0, jos haluat laskea lainan pituuden maksuerän perusteella.
- maksuerä: aseta maksueräksi 0, jos haluat laskea lainan maksuerän lainan pituuden perusteella.

In [3]:
aloituspäivä = date(2026, 1, 1)
laina = 300000
korkokanta = .0245
aika_kuukausina = 0
maksuerä = 1500

## Lyhennystaulukon luonti

Tässä luodaan lyhennystaulukko **annuiteetti.py**-moduulin **lyhennystaulukko**-funktiolla.

In [4]:
taulukko, tiedot = annuiteetti.lyhennystaulukko(aloituspäivä, laina, korkokanta, aika_kuukausina, maksuerä)

## Vertailun lähtötiedot

- aloituspäivä on sama kaikille vertailtaville lainoille.
- lainat, korkokannat, ajat_kuukausina, maksuerät: kirjoita jokaiseen yhtä monta vaihtoehtoa. Vaihtoehtoja voit kirjoittaa enemmänkin kuin kolme.
- jos ajat_kuukausina=0, niin vastaavan maksuerän pitää olla nollaa suurempi.
- jos maksuerät=0, niin vastaavan ajat_kuukausina pitää olla nollaa suurempi.

In [5]:
aloituspäivä = date(2026, 1, 1)
lainat = (300000, 300000, 300000)
korkokannat = (.0245, .0245, .0245)
ajat_kuukausina = (0, 0, 0)
maksuerät = (1300, 1500, 1700)

## Vertailu-taulukon luonti

Tässä luodaan vertailu-taulukko **annuiteetti.py**-moduulin **lyhennystaulukko**-funktion avulla.

In [6]:
vertailu = pd.DataFrame(columns = ['Lainan suuruus', 'Korkokanta', 'Aika kuukausina', 'Maksuerä', 
                                   'Korot yhteensä', 'Viimeinen maksuerä'])
for i in range(len(lainat)):    
    v_taulukko, v_tiedot = annuiteetti.lyhennystaulukko(aloituspäivä=aloituspäivä, laina=lainat[i], korkokanta=korkokannat[i], 
                                            aika_kuukausina=ajat_kuukausina[i], maksuerä=maksuerät[i])
    vertailu.loc[i] = v_tiedot.T

## Lyhennystaulukko ja vertailu-taulukko Excel-tiedostoon

Seuraavassa kirjoitetaan Excel-tiedostoon **pandas**-paketin **to_excel**-funktiolla käyttäen kirjoittimena **xlsxwriter**ia . Huomaa, että kirjoittaminen Excel-tiedostoon on teknisesti eri asia kuin tietojen kirjoittaminen auki olevaan Exceliin. **xlwings**-paketin toimintoja ei siis voi käyttää Excel-tiedostoon kirjoittamiseen.

**xlsxwriter** on monipuolinen ja hyvin dokumentoitu paketti. Katso https://xlsxwriter.readthedocs.io/

In [7]:
# xlsxwriter-kirjoittimen luonti
writer = pd.ExcelWriter('laina.xlsx', engine='xlsxwriter')

# lyhennystaulukon kirjoittaminen lyhennystaulukko-välilehdelle
font_bold = writer.book.add_format({'bold':True})
rivi = 2
tiedot.to_excel(writer, sheet_name='lyhennystaulukko', startrow=rivi)
taulukko.to_excel(writer, sheet_name='lyhennystaulukko', index=False, startrow=rivi+tiedot.shape[0]+2)
writer.sheets['lyhennystaulukko'].write('A1', 'Annuiteettilainan lyhennystaulukko', font_bold)
writer.sheets['lyhennystaulukko'].autofit()

# Vertailu-taulukon kirjoittaminen vertailu-välilehdelle
vertailu.to_excel(writer, sheet_name='vertailu', index=False, startrow=rivi)
writer.sheets['vertailu'].write('A1', 'Vaihtoehtojen vertailu', font_bold)
writer.sheets['vertailu'].autofit()
writer.close()